In [2]:
import os
import pandas as pd
import numpy as np

from loguru import logger
from tracks_import import read_from_csv

In [3]:
# Specify the paths and track id
dataset_dir = '../../data/raw/inD/data/'
recording = "00"


if not os.path.isdir(dataset_dir):
    raise Exception("Directory does not exist")


# Create paths to csv files
tracks_file = dataset_dir + recording + "_tracks.csv"
tracks_meta_file = dataset_dir + recording + "_tracksMeta.csv"
recording_meta_file = dataset_dir + recording + "_recordingMeta.csv"

if not os.path.isfile(tracks_file) or not os.path.isfile(tracks_meta_file) or not os.path.isfile(recording_meta_file):
    raise Exception("File does not exist")
# Load csv files
logger.info("Loading csv files {}, {} and {}", tracks_file, tracks_meta_file, recording_meta_file)
tracks, static_info, meta_info = read_from_csv(tracks_file, tracks_meta_file, recording_meta_file,
                                                   include_px_coordinates=True)
df_tracks = pd.DataFrame(tracks)
df_static_info = pd.DataFrame(static_info)
df_meta_info = pd.DataFrame(meta_info.items())

2023-12-28 12:38:25.630 | INFO     | __main__:<module>:18 - Loading csv files ../../data/raw/inD/data/00_tracks.csv, ../../data/raw/inD/data/00_tracksMeta.csv and ../../data/raw/inD/data/00_recordingMeta.csv


In [4]:
# Select the track(s)
n = 1
track_ids = list(range(n))  

selected_tracks = df_tracks[df_tracks['trackId'].isin(track_ids)]

# Change the position to the distance by subtracting the starting position from the position lists
selected_tracks.loc[:, 'xCenter'] = selected_tracks['xCenter'].apply(lambda num: [x - num[0] for x in num[:]])
selected_tracks.loc[:, 'yCenter'] = selected_tracks['yCenter'].apply(lambda num: [x - num[0] for x in num[:]])

# Select distance, velocity, and acceleration columns
columns_to_select = ['xCenter', 'yCenter', 'xVelocity', 'yVelocity', 'xAcceleration', 'yAcceleration']
selected_data = selected_tracks[columns_to_select]

# Unstack the selected_data DataFrame to flatten the lists in the 'xCenter', 'yCenter', etc. columns
selected_data_unstacked = selected_data.apply(lambda x: np.concatenate(x.values))

# Create a new DataFrame
new_dataframe = pd.DataFrame(selected_data_unstacked)
new_dataframe


,xCenter,yCenter,xVelocity,yVelocity,xAcceleration,yAcceleration
0,0.00000,0.00000,-1.20819,-9.06651,-0.01047,0.52307
1,-0.04689,-0.35613,-1.20844,-9.06017,-0.01211,0.55970
2,-0.09698,-0.71906,-1.20806,-9.05198,-0.01427,0.61071
3,-0.14658,-1.08403,-1.20726,-9.04039,-0.01754,0.67524
4,-0.19546,-1.44961,-1.20636,-9.02442,-0.02230,0.75172
...,...,...,...,...,...,...
273,-68.20249,-65.38038,-13.68996,-8.46034,-1.39073,-0.77769
274,-68.75107,-65.71988,-13.74591,-8.49028,-1.39115,-0.77614
275,-69.30231,-66.05907,-13.80176,-8.52057,-1.39130,-0.77554
276,-69.85589,-66.39917,-13.85746,-8.55137,-1.39133,-0.77543


In [5]:
# Assuming 'new_dataframe' is your DataFrame
output_directory = '../../data/processed/'
file_name = f'x_y_recording{recording}_range{n}.csv'
output_file_path = os.path.join(output_directory, file_name)

# Save the new_dataframe as a CSV file in the data/processed directory
new_dataframe.to_csv(output_file_path, index=False)
